The code in utilsGRF/writescripts.py can be used to write the files with C++ functions to compute position and steepness for a given graph, as well as mathematica files to double check the results. C++ code is to be executed from python using pybind (install with pip install pybind11).

Examples of writing and function calling are given here.


In [1]:
import numpy as np
import sys, os
sys.path.append('../utilsGRF/')
import writescripts 

# 1. Simple equilibrium TF binding models

Use class PrepareFilesEqbindingmodels for models at equilibrium.

## all-or-none

In [2]:
#write files
#
obj=writescripts.PrepareFilesEqbindingmodels(varGRF='x',concvars=['x'],N=4,strategy='an') #4 binding sites, all-or-nothing strategy.
obj.get_rhos_and_GRFs() #this computes products along paths and GRF

folder='./TFbinding'
if not os.path.isdir(folder):
    os.mkdir(folder)
fname=os.path.join(folder,'N4an.cpp') #N4an will be the name of the pybind module as well
typestring='long double' 
funcname='GRF_N4an'
obj.write_pybind_interface(fname=fname, funcname=funcname, typestring=typestring)
obj.write_checkfile_mathematica_singlevar(fname=fname.replace('.cpp','.wl')) #this is to check the results of the boundary search (see 2019_11_11_analyseboundries for creating the input files)
obj.write_checksingleparset_mathematica_singlevar(fname=fname.replace('.cpp','.nb')) #this is to check an individual parameter set. In this case, copy-paste the parameter set into PLIST={} in the mathematica notebook

#compile. Notice that this requires eigen, which only needs to be downloaded to a desired location.
#Modify path accordingly. On a mac, use -undefined dynamic_lookup
filename=fname
path_1,file=os.path.split(fname)
#objectnamelinux=os.path.join(path_1,'cflinux',file.replace('.cpp',''))
#objectnamemac=os.path.join(path_1,'cfmac',file.replace('.cpp',''))
objectnamemac=os.path.join(path_1,file.replace('.cpp','')) 
#compilestringlinux="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -I /home/rosamc/eigenlibrary/eigen-git-mirror -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamelinux)
compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamemac)
compilestring=compilestringmac
print(compilestring)

!$compilestring

['K1', 'K2', 'K3', 'K4', 'w12', 'w13', 'w14', 'w23', 'w24', 'w34', 'w123', 'w124', 'w134', 'w234', 'w1234']
parlist is ['K1', 'K2', 'K3', 'K4', 'w12', 'w13', 'w14', 'w23', 'w24', 'w34', 'w123', 'w124', 'w134', 'w234', 'w1234']
GRF numerator:1*(rho16)
GRF denominator:1*(rho1+rho2+rho3+rho4+rho5+rho6+rho7+rho8+rho9+rho10+rho11+rho12+rho13+rho14+rho15+rho16)
0
1
writing mathematica file
K1_,K2_,K3_,K4_,w12_,w13_,w14_,w23_,w24_,w34_,w123_,w124_,w134_,w234_,w1234_,
writing mathematica file
K1_,K2_,K3_,K4_,w12_,w13_,w14_,w23_,w24_,w34_,w123_,w124_,w134_,w234_,w1234_,
c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` ./TFbinding/N4an.cpp -o ./TFbinding/N4an`python3-config --extension-suffix`
In file included from ./TFbinding/N4an.cpp:12:
In file included from ../utilsGRF/posstpfunc_cpp_longdouble.h:7:
../utilsGRF/unipolynoboost.hpp:796:22: war

In [3]:
#Now load the created pybind module and test position-steepness computation:
sys.path.append(folder) #add the folder where the module has been created to the path
import N4an #import pybind module created in the cell above


np.random.seed(3)
parset=10**np.random.uniform(-3,3,size=15)
for par in parset:
    print(par,end=',')
print(' ')
    
print(N4an.interfaceps_a_GRF_N4an_x(parset)) #compute position steepness using aberth method for roots of polynomial
print(N4an.interfaceps_s_GRF_N4an_x(parset)) #compute position steepness using eigenvalues of companion matrix. 
print(N4an.interface_GRF_N4an_x(parset,50)) #compute GRF value at x=50
print(N4an.interfacemonotonic_GRF_N4an_x(parset)) #checks for monotonicity according to whether there are critical points int the GRF. 
#Careful, the monotonicity check has not been tested. 

2.0173786271782266,17.737276482506807,0.05564529299329967,1.1613567037801702,227.8671529202063,238.64840361732638,0.005669070564161234,0.01751690073164394,0.002036119296609855,0.4414271712959385,0.001510974953167781,0.5508068674169324,7.849902917177079,0.04687310204114261,11.416406177374222, 
[0.57467288 0.64913956]
[0.57467288 0.64913956]
0.14265421644724108
1


In [4]:
#now an example with only pairwise cooperativities, to compare with code Felix sent some time ago
parset=np.ones(15)
parset[0:10]=[2.088244750240813, 1.3463954822067459, 3.285450776234611, 1.5839516209052389, 2.7744710972974485, 1.7690548476943588, 3.016389950508736, 880.6497676758913, 0.01891313961169344, 0.002484648069946382]
print(N4an.interfaceps_a_GRF_N4an_x(parset)) #compute position steepness using aberth method for roots of polynomial
print(N4an.interfaceps_s_GRF_N4an_x(parset)) #compute position steepness using eigenvalues of companion matrix. Fails
print(N4an.interface_GRF_N4an_x(parset,50000)) #compute GRF value at x=50000

[1.64546974e-04 9.99510837e-01]
[-1. -1.]
0.18263312481093405


## average binding

In [5]:
#write files
obj=writescripts.PrepareFilesEqbindingmodels(varGRF='x',concvars=['x'],N=5,strategy='av')
obj.get_rhos_and_GRFs()

folder='./TFbinding'
if not os.path.isdir(folder):
    os.mkdir(folder)
fname=os.path.join(folder,'N5av.cpp') #N4an will be the name of the pybind module as well
typestring='long double' 
funcname='GRF_N5av'
obj.write_pybind_interface(fname=fname, funcname=funcname, typestring=typestring)
obj.write_checkfile_mathematica_singlevar(fname=fname.replace('.cpp','.wl'))
obj.write_checksingleparset_mathematica_singlevar(fname=fname.replace('.cpp','.nb'))

#compile. Notice that this requires eigen. Modify path accordingly. On a mac, use -undefined dynamic_lookup
filename=fname
path_1,file=os.path.split(fname)
#objectnamelinux=os.path.join(path_1,'cflinux',file.replace('.cpp',''))
#objectnamemac=os.path.join(path_1,'cfmac',file.replace('.cpp',''))
objectnamemac=os.path.join(path_1,file.replace('.cpp',''))
#compilestringlinux="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -I /home/rosamc/eigenlibrary/eigen-git-mirror -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamelinux)
compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamemac)
compilestring=compilestringmac
print(compilestring)

!$compilestring

['K1', 'K2', 'K3', 'K4', 'K5', 'w12', 'w13', 'w14', 'w15', 'w23', 'w24', 'w25', 'w34', 'w35', 'w45', 'w123', 'w124', 'w125', 'w134', 'w135', 'w145', 'w234', 'w235', 'w245', 'w345', 'w1234', 'w1235', 'w1245', 'w1345', 'w2345', 'w12345']
parlist is ['K1', 'K2', 'K3', 'K4', 'K5', 'w12', 'w13', 'w14', 'w15', 'w23', 'w24', 'w25', 'w34', 'w35', 'w45', 'w123', 'w124', 'w125', 'w134', 'w135', 'w145', 'w234', 'w235', 'w245', 'w345', 'w1234', 'w1235', 'w1245', 'w1345', 'w2345', 'w12345']
GRF numerator:(1.0/5)*(rho2)+(1.0/5)*(rho3)+(1.0/5)*(rho4)+(1.0/5)*(rho5)+(1.0/5)*(rho6)+(2.0/5)*(rho7)+(2.0/5)*(rho8)+(2.0/5)*(rho9)+(2.0/5)*(rho10)+(2.0/5)*(rho11)+(2.0/5)*(rho12)+(2.0/5)*(rho13)+(2.0/5)*(rho14)+(2.0/5)*(rho15)+(2.0/5)*(rho16)+(3.0/5)*(rho17)+(3.0/5)*(rho18)+(3.0/5)*(rho19)+(3.0/5)*(rho20)+(3.0/5)*(rho21)+(3.0/5)*(rho22)+(3.0/5)*(rho23)+(3.0/5)*(rho24)+(3.0/5)*(rho25)+(3.0/5)*(rho26)+(4.0/5)*(rho27)+(4.0/5)*(rho28)+(4.0/5)*(rho29)+(4.0/5)*(rho30)+(4.0/5)*(rho31)+(5.0/5)*(rho32)
GRF denominator

In [6]:
#Now load the created pybind module and test position-steepness computation:
sys.path.append('./TFbinding')
import N5av 

parlist=['K1', 'K2', 'K3', 'K4', 'K5', 'w12', 'w13', 'w14', 'w15', 'w23', 'w24', 'w25', 'w34', 'w35', 'w45', 'w123', 'w124', 'w125', 'w134', 'w135', 'w145', 'w234', 'w235', 'w245', 'w345', 'w1234', 'w1235', 'w1245', 'w1345', 'w2345', 'w12345']
npars=len(parlist)

np.random.seed(3)
parset=10**np.random.uniform(-3,3,size=npars)
for par in parset:
    print(par,end=',')
print(' ')
    
print(N5av.interfaceps_a_GRF_N5av_x(parset)) #compute position steepness using aberth method for roots of polynomial
print(N5av.interfaceps_s_GRF_N5av_x(parset)) #compute position steepness using eigenvalues of companion matrix. Fails
print(N5av.interface_GRF_N5av_x(parset,0.015)) #compute GRF value at x=50



2.0173786271782266,17.737276482506807,0.05564529299329967,1.1613567037801702,227.8671529202063,238.64840361732638,0.005669070564161234,0.01751690073164394,0.002036119296609855,0.4414271712959385,0.001510974953167781,0.5508068674169324,7.849902917177079,0.04687310204114261,11.416406177374222,3.508947775032447,0.0013928081335714791,2.254885676794552,0.03593475443929519,0.30946189602226354,0.05025166901715467,14.415426957087972,0.43926065620578164,0.008733864117847177,1.8530797413430484,48.07160108233731,0.06889396724912579,0.021465811117829078,0.21272941477264604,415.24259760203694,717.7488987687258, 
[0.9640814  0.64565983]
[0.9640814  0.64565983]
0.4401038727497043


# 2. Equilibrium TF binding with Pol 

In [7]:
obj=writescripts.PrepareFilesEqbindingmodels(varGRF='x',concvars=['x','P'],N=5,strategy='pol')
obj.get_rhos_and_GRFs()

folder='./TFbindingP'
if not os.path.isdir(folder):
    os.mkdir(folder)
fname=os.path.join(folder,'N5P.cpp') #N4an will be the name of the pybind module as well
typestring='long double' 
funcname='GRF_N5P'


obj.write_pybind_interface(fname=fname, funcname=funcname, typestring='long double')
obj.write_checkfile_mathematica_singlevar(fname=fname.replace('.cpp','.wl'),additionallinespars="P=1;\n")
obj.write_checksingleparset_mathematica_singlevar(fname=fname.replace('.cpp','.nb'),additionallinespars="P=1;\n")

['K1', 'K2', 'K3', 'K4', 'K5', 'w12', 'w13', 'w14', 'w15', 'w23', 'w24', 'w25', 'w34', 'w35', 'w45', 'w123', 'w124', 'w125', 'w134', 'w135', 'w145', 'w234', 'w235', 'w245', 'w345', 'w1234', 'w1235', 'w1245', 'w1345', 'w2345', 'w12345', 'wp1', 'wp2', 'wp3', 'wp4', 'wp5', 'wp12', 'wp13', 'wp14', 'wp15', 'wp23', 'wp24', 'wp25', 'wp34', 'wp35', 'wp45', 'wp123', 'wp124', 'wp125', 'wp134', 'wp135', 'wp145', 'wp234', 'wp235', 'wp245', 'wp345', 'wp1234', 'wp1235', 'wp1245', 'wp1345', 'wp2345', 'wp12345']
parlist is ['K1', 'K2', 'K3', 'K4', 'K5', 'w12', 'w13', 'w14', 'w15', 'w23', 'w24', 'w25', 'w34', 'w35', 'w45', 'w123', 'w124', 'w125', 'w134', 'w135', 'w145', 'w234', 'w235', 'w245', 'w345', 'w1234', 'w1235', 'w1245', 'w1345', 'w2345', 'w12345', 'wp1', 'wp2', 'wp3', 'wp4', 'wp5', 'wp12', 'wp13', 'wp14', 'wp15', 'wp23', 'wp24', 'wp25', 'wp34', 'wp35', 'wp45', 'wp123', 'wp124', 'wp125', 'wp134', 'wp135', 'wp145', 'wp234', 'wp235', 'wp245', 'wp345', 'wp1234', 'wp1235', 'wp1245', 'wp1345', 'wp234

In [8]:
#compile. Notice that this requires eigen. Modify path accordingly. On a mac, use -undefined dynamic_lookup
filename=fname
path_1,file=os.path.split(fname)
#objectnamelinux=os.path.join(path_1,'cflinux',file.replace('.cpp',''))
#objectnamemac=os.path.join(path_1,'cfmac',file.replace('.cpp',''))
objectnamemac=os.path.join(path_1,file.replace('.cpp',''))
#compilestringlinux="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -I /home/rosamc/eigenlibrary/eigen-git-mirror -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamelinux)
compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamemac)
compilestring=compilestringmac
print(compilestring)

!$compilestring

c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` ./TFbindingP/N5P.cpp -o ./TFbindingP/N5P`python3-config --extension-suffix`
In file included from ./TFbindingP/N5P.cpp:12:
In file included from ../utilsGRF/posstpfunc_cpp_longdouble.h:7:
../utilsGRF/unipolynoboost.hpp:796:22: warning: unused variable 'degree'
      [-Wunused-variable]
            unsigned degree = this->coefs.size() - 1;
                     ^
../utilsGRF/unipolynoboost.hpp:814:22: warning: unused variable 'degree'
      [-Wunused-variable]
            unsigned degree = (this->coefs).size() - 1;
                     ^
In file included from ./TFbindingP/N5P.cpp:12:
../utilsGRF/posstpfunc_cpp_longdouble.h:28:17: warning: unused variable 'x05'
      [-Wunused-variable]
    long double x05;
                ^
../utilsGRF/posstpfunc_cpp_longdouble.h:51:43: warning: unused var

In [9]:
sys.path.append('./TFbindingP')
import N5P
parset=np.array([1.0, 4.584787857485738, 549.4700341183042, 0.001,0.001914248655777849, 0.001, 0.001, 0.001, 12.294644026152989,0.23185107756655418, 0.9460054003327282, 42.75443709622185,0.00761927921243007, 32.80795579727805, 1000.0, 0.42446221098782855,0.06015394290338065, 7.14231047037886, 5.715956452401879,0.40125545915056343, 1.3764246977240986, 1000.0, 0.21039595242238818,0.03639906644469741, 1000.0, 0.10970731837325615,0.009439957524295297, 0.001, 8.8809885439463, 470.4551782166511,1000.0, 300.3180209647626, 3.704521914955922, 1.0, 37.80105790657998,141.38708832585937, 300.3180209647626, 300.3180209647626, 1000.0,1000.0, 8.248435051715608, 1000.0, 141.38708832585937,37.80105790657998, 141.38708832585937, 141.38708832585937,300.3180209647626, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0,1000.0, 1000.0, 156.56176820296534, 1000.0, 1000.0, 1000.0, 1000.0,1000.0, 1000.0])
#parset=np.array([1.0, 4.584787857485738, 549.4700341183042, 0.001,0.001914248655777849, 0.001, 0.001, 0.001, 12.294644026152989,0.23185107756655418, 0.9460054003327282, 42.75443709622185,0.00761927921243007, 32.80795579727805, 1000.0, 0.42446221098782855,0.06015394290338065, 7.14231047037886, 5.715956452401879,0.40125545915056343, 1.3764246977240986, 1000.0, 0.21039595242238818,0.03639906644469741, 1000.0, 0.10970731837325615,0.009439957524295297, 0.001, 8.8809885439463, 470.4551782166511,1000.0, 300.3180209647626, 3.704521914955922, 1.0, 37.80105790657998,141.38708832585937, 300.3180209647626, 300.3180209647626, 1000.0,1000.0, 8.248435051715608, 1000.0, 141.38708832585937,37.80105790657998, 141.38708832585937, 141.38708832585937,300.3180209647626, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0,1000.0, 1000.0, 156.56176820296534, 1000.0, 1000.0, 1000.0, 100.0,100.0, 100.0])
pol=np.array([1])
print(N5P.interfaceps_a_GRF_N5P_x(parset,pol))
print(N5P.interface_GRF_N5P_x(parset,pol,0.01))



[1.2844962  0.40903896]
0.9706023321150515


# 3. TF binding graph away from equilibrium

In this case use class PrepareFilesNoneq. 
This requires the edges of the graph as a list of tuples, where each tuple is (index source node, label, index target node) e.g. (1, 'k1', 2). TF should be indicated where appropriate with a dash e.g. (1, 'a1-x', 2). One can use any variable for this, just make sure it is appropriately indicated in varGRF and concvars. 

In [10]:
import generatehypercube

#In this case the edges are obtained from a graph generated with the code in utilsGRF/generatehypercube
G=generatehypercube.hypercube(3) #3 is the number of binding sites
edges=[]
for edge in G.edges:
    #print(edge,G2[edge[0]][edge[1]]['weight'])
    tuple_=(edge[0],G[edge[0]][edge[1]]['k'],edge[1])
    edges.append(tuple_)

print(edges)
parlist=[x[1] for x in edges]
for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
    
print(parlist)

MTTfolder='../utilsGRF/' #folder with MTT.py script
basename='graph' #name (with no extention) of the file of the graph that will be read by MTT.py

obj=writescripts.PrepareFilesNoneq(edgelist=edges,varGRF='x',concvars=['x'],parlist=parlist,MTTfolder=MTTfolder,graphbasename=basename)
obj.write_execute_parse()
obj.simpify_rhos()

#manually define the expressions for the numerator and the denominator in terms of the rhos. Indexing respects breadth-first traversal
#all-or-nothing strategy
num="1*(rho8)" #Be sure to indicate even if multiplied by 1.
den="1*(rho1+rho2+rho3+rho4+rho5+rho6+rho7+rho8)" #Be sure to indicate even if multiplied by 1.

obj.parse_GRF(num,den)


folder='./TFbindingnoneq'
if not os.path.isdir(folder):
    os.mkdir(folder)
fname=os.path.join(folder,'N3an_neq.cpp') #N4an will be the name of the pybind module as well
typestring='long double' 
funcname='GRF_N3an_neq'


obj.write_pybind_interface(fname=fname, funcname=funcname, typestring='long double')
obj.write_checkfile_mathematica_singlevar(fname=fname.replace('.cpp','.wl'))
obj.write_checksingleparset_mathematica_singlevar(fname=fname.replace('.cpp','.nb'))

[(1, 'a1e-x', 2), (1, 'a2e-x', 3), (1, 'a3e-x', 4), (2, 'b11', 1), (2, 'a21-x', 5), (2, 'a31-x', 6), (5, 'b212', 2), (5, 'a312-x', 8), (5, 'b112', 3), (8, 'b3123', 5), (8, 'b2123', 6), (8, 'b1123', 7), (6, 'b313', 2), (6, 'a213-x', 8), (6, 'b113', 4), (3, 'b22', 1), (3, 'a12-x', 5), (3, 'a32-x', 7), (7, 'b323', 3), (7, 'a123-x', 8), (7, 'b223', 4), (4, 'b33', 1), (4, 'a13-x', 6), (4, 'a23-x', 7)]
['a1e', 'a2e', 'a3e', 'b11', 'a21', 'a31', 'b212', 'a312', 'b112', 'b3123', 'b2123', 'b1123', 'b313', 'a213', 'b113', 'b22', 'a12', 'a32', 'b323', 'a123', 'b223', 'b33', 'a13', 'a23']
removing  ../utilsGRF/graph-parsed.txt
removing  ../utilsGRF/graph-6.txt
removing  ../utilsGRF/graph-7.txt
removing  ../utilsGRF/graph-5.txt
removing  ../utilsGRF/graph-4.txt
removing  ../utilsGRF/graph-1.txt
removing  ../utilsGRF/graph-3.txt
removing  ../utilsGRF/graph-2.txt
removing  ../utilsGRF/graph-8.txt
removing  ../utilsGRF/graph.txt
executing MTT
0
0
1
writing mathematica file
a1e_,a2e_,a3e_,b11_,a21_,a31

In [11]:
filename=fname
path_1,file=os.path.split(fname)
#objectnamelinux=os.path.join(path_1,'cflinux',file.replace('.cpp',''))
#objectnamemac=os.path.join(path_1,'cfmac',file.replace('.cpp',''))
objectnamemac=os.path.join(path_1,file.replace('.cpp',''))
#compilestringlinux="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -I /home/rosamc/eigenlibrary/eigen-git-mirror -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamelinux)
compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamemac)
compilestring=compilestringmac
print(compilestring)

!$compilestring

c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` ./TFbindingnoneq/N3an_neq.cpp -o ./TFbindingnoneq/N3an_neq`python3-config --extension-suffix`
In file included from ./TFbindingnoneq/N3an_neq.cpp:12:
In file included from ../utilsGRF/posstpfunc_cpp_longdouble.h:7:
../utilsGRF/unipolynoboost.hpp:796:22: warning: unused variable 'degree'
      [-Wunused-variable]
            unsigned degree = this->coefs.size() - 1;
                     ^
../utilsGRF/unipolynoboost.hpp:814:22: warning: unused variable 'degree'
      [-Wunused-variable]
            unsigned degree = (this->coefs).size() - 1;
                     ^
In file included from ./TFbindingnoneq/N3an_neq.cpp:12:
../utilsGRF/posstpfunc_cpp_longdouble.h:28:17: warning: unused variable 'x05'
      [-Wunused-variable]
    long double x05;
                ^
../utilsGRF/posstpfunc_cpp_lon

In [12]:
sys.path.append('./TFbindingnoneq/')
import N3an_neq
parset=np.array([1.0,403.17,0.001372,1.0,915.51,0.0078561,840.66,44.214,0.71468,0.001067,16.253,0.0034346,0.0013987,903.31,1.2721,657.75,148.47,0.29436,0.0044066,975.57,274.17,0.26978,5.6676,972.82,])

In [13]:
N3an_neq.interfaceps_a_GRF_N3an_neq_x(parset)

array([0.82997169, 1.23461412])

# 2 conformation graph, away from equilibrium

In [14]:
MTTfolder='../utilsGRF/' #folder with MTT.py 
basename='graph'

#in this case I am writing the edges manually. Be sure to start labelling the nodes by 1 (not 0), or MTT.py will fail
edges=[(1,'k_1_1_0-x',2),(2,'km_1_1_1',1),
       (1,'k_1_2_0-x',3),(3,'km_1_2_2',1),
       (2,'k_1_2_1-x',4), (4,'km_1_2_12',2),
       (3,'k_1_1_2-x',4), (4,'km_1_1_12',3),
       (5,'k_2_1_0-x',6),(6,'km_2_1_1',5),
       (5,'k_2_2_0-x',7),(7,'km_2_2_2',5),
       (6,'k_2_2_1-x',8), (8,'km_2_2_12',6),
       (7,'k_2_1_2-x',8), (8,'km_2_1_12',7),
       (1,'l_1t2_0',5),(5,'l_2t1_0',1),
       (2,'l_1t2_1',6),(6,'l_2t1_1',2),
       (3,'l_1t2_2',7),(7,'l_2t1_2',3),
       (4,'l_1t2_12',8),(8,'l_2t1_12',4),
       ]
parlist=[x[1] for x in edges]
for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
print(parlist)

obj=writescripts.PrepareFilesNoneq(edgelist=edges,varGRF='x',concvars=['x'],parlist=parlist,MTTfolder=MTTfolder,graphbasename=basename)
obj.write_execute_parse()
obj.simpify_rhos()
num="0.5*(rho2+rho3+rho6+rho7)+1*(rho4+rho8)" #Be sure to indicate even if multiplied by 1.
den="1*(rho1+rho2+rho3+rho4+rho5+rho6+rho7+rho8)" #Be sure to indicate even if multiplied by 1.
obj.parse_GRF(num,den)

#For small graphs, this can also be done instead of previous line
#rhos=obj.all_rhos
#denstring=''
#for rnum, rho in enumerate(rhos):
#    if rnum<len(rhos)-1:
#        denstring+=rho.split('=')[1]+'+'
#    else:
#        denstring+=rho.split('=')[1]
#numstring='(1.0/2)*(%s+%s+%s+%s)+1*(%s+%s)'%tuple([x.split('=')[1] for x in [rhos[1],rhos[2],rhos[5],rhos[6],rhos[3],rhos[7]]])
#obj.simpify_num_den(numstring,denstring)

folder='./btwoconf'
if not os.path.isdir(folder):
    os.mkdir(folder)
fname=os.path.join(folder,'bs2c2av.cpp') #N4an will be the name of the pybind module as well
typestring='long double' 
funcname='GRF_bs2c2av'


obj.write_pybind_interface(fname=fname, funcname=funcname, typestring='long double')
obj.write_checkfile_mathematica_singlevar(fname=fname.replace('.cpp','.wl'))
obj.write_checksingleparset_mathematica_singlevar(fname=fname.replace('.cpp','.nb'))

['k_1_1_0', 'km_1_1_1', 'k_1_2_0', 'km_1_2_2', 'k_1_2_1', 'km_1_2_12', 'k_1_1_2', 'km_1_1_12', 'k_2_1_0', 'km_2_1_1', 'k_2_2_0', 'km_2_2_2', 'k_2_2_1', 'km_2_2_12', 'k_2_1_2', 'km_2_1_12', 'l_1t2_0', 'l_2t1_0', 'l_1t2_1', 'l_2t1_1', 'l_1t2_2', 'l_2t1_2', 'l_1t2_12', 'l_2t1_12']
removing  ../utilsGRF/graph-parsed.txt
removing  ../utilsGRF/graph-6.txt
removing  ../utilsGRF/graph-7.txt
removing  ../utilsGRF/graph-5.txt
removing  ../utilsGRF/graph-4.txt
removing  ../utilsGRF/graph-1.txt
removing  ../utilsGRF/graph-3.txt
removing  ../utilsGRF/graph-2.txt
removing  ../utilsGRF/graph-8.txt
removing  ../utilsGRF/graph.txt
executing MTT
0
0
1
writing mathematica file
kU1U1U0_,kmU1U1U1_,kU1U2U0_,kmU1U2U2_,kU1U2U1_,kmU1U2U12_,kU1U1U2_,kmU1U1U12_,kU2U1U0_,kmU2U1U1_,kU2U2U0_,kmU2U2U2_,kU2U2U1_,kmU2U2U12_,kU2U1U2_,kmU2U1U12_,lU1t2U0_,lU2t1U0_,lU1t2U1_,lU2t1U1_,lU1t2U2_,lU2t1U2_,lU1t2U12_,lU2t1U12_,
writing mathematica file
kU1U1U0_,kmU1U1U1_,kU1U2U0_,kmU1U2U2_,kU1U2U1_,kmU1U2U12_,kU1U1U2_,kmU1U1U12_

In [15]:
filename=fname
path_1,file=os.path.split(fname)
#objectnamelinux=os.path.join(path_1,'cflinux',file.replace('.cpp',''))
#objectnamemac=os.path.join(path_1,'cfmac',file.replace('.cpp',''))
objectnamemac=os.path.join(path_1,file.replace('.cpp',''))
#compilestringlinux="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -I /home/rosamc/eigenlibrary/eigen-git-mirror -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamelinux)
compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(filename,objectnamemac)
compilestring=compilestringmac
print(compilestring)

!$compilestring

c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -I ../utilsGRF  `python3 -m pybind11 --includes` ./btwoconf/bs2c2av.cpp -o ./btwoconf/bs2c2av`python3-config --extension-suffix`
In file included from ./btwoconf/bs2c2av.cpp:12:
In file included from ../utilsGRF/posstpfunc_cpp_longdouble.h:7:
../utilsGRF/unipolynoboost.hpp:796:22: warning: unused variable 'degree'
      [-Wunused-variable]
            unsigned degree = this->coefs.size() - 1;
                     ^
../utilsGRF/unipolynoboost.hpp:814:22: warning: unused variable 'degree'
      [-Wunused-variable]
            unsigned degree = (this->coefs).size() - 1;
                     ^
In file included from ./btwoconf/bs2c2av.cpp:12:
../utilsGRF/posstpfunc_cpp_longdouble.h:28:17: warning: unused variable 'x05'
      [-Wunused-variable]
    long double x05;
                ^
../utilsGRF/posstpfunc_cpp_longdouble.h:51:43: warning: un

In [16]:
sys.path.append(folder)
import bs2c2av
parset=np.array([0.3177840006884067,20.986835607646604,0.1001581395585897,6.516215458215692,0.0075951323286823896,0.0035812246787002297,0.013108749615263331,0.11840345146135145,0.24021761202431605,1.7096232052870348,0.327466657122539,12.921621108432518,0.01685440782816938,185.65413149854675,0.0014599082378876644,10.539137268289434,0.31902800949575993,2.2497744000332283,0.006955392321661601,0.015438636235829037,63.74812448657215,645.0144652189368,0.07595302389085043,14.253965195124682])
bs2c2av.interfaceps_a_GRF_bs2c2av_x(parset)

array([0.13664639, 0.60488303])

In [ ]:
#This gives segmentation fault. Check what is going on!!
parset=np.array([3.5388446613421465,64.15165258111196,3.0182085913100694,324.558501648249,879.8096935488961,0.018966777988799855,4.108760050664538,0.0040084948761643785,0.001,0.001,0.20220978144309498,0.001,0.15586197045854797,0.001,0.001,0.00236406093526841,0.001,0.001,1.28639208082208,1.3720640905985517,0.001,0.0019041560326281237,0.468181892817541,0.001])
bs2c2av.interfaceps_a_GRF_bs2c2av_x(parset)